In [58]:
# BÀI 1: Hãy viết câu lệnh SQL để tính sự tương quan giữa A và B theo công thức 
import pandas as pd
import numpy as np
# Tạo DataFrame với giá trị ngẫu nhiên từ 1 đến 20, gồm 10 hàng
df = pd.DataFrame({
    'A': np.random.randint(1, 21, size=10),  # random integers từ 1 đến 10
    'B': np.random.randint(1, 21, size=10)
})


n = len(df)
sum_ab = (df['A'] * df['B']).sum()
sum_a = df['A'].sum()
sum_b = df['B'].sum()
sum_a2 = (df['A'] ** 2).sum()
sum_b2 = (df['B'] ** 2).sum()

# Áp dụng công thức Pearson
numerator = n * sum_ab - sum_a * sum_b
denominator = np.sqrt((n * sum_a2 - sum_a ** 2) * (n * sum_b2 - sum_b ** 2))

correlation = numerator / denominator

print(f"Hệ số tương quan Pearson giữa A và B là: {correlation:4f}")

Hệ số tương quan Pearson giữa A và B là: 0.306190


In [59]:
# BÀI 2 Một công ty oto đang kiểm tra 3 loại mẫu mới A, B và C trong 4 ngày
#  và chấm điểm theo thang từ 1đến 10 điểm cho mỗi ngày với bảng sau. Liệu có sự khác biệt đáng kể giữa các mẫu dựa trên điểm số mà chúng nhận được trong 4 ngày thử nghiệm không?
#  Kết quả thử nghiệm phụ thuộc vào ngày hay phụ thuộc vào mẫu xe? Hãy chuyển đổi dữ liệu sang dạng quan hệ và thực hiện kiểm tra χ2
import pandas as pd
from scipy.stats import chi2_contingency

#1 Tạo DataFrame từ bảng
df = pd.DataFrame({
    'Day': ['Day 1', 'Day 2', 'Day 3', 'Day 4'],
    'A': [8, 7.5, 6, 7],
    'B': [9, 8.5, 7, 6],
    'C': [7, 7, 8, 5]
})
# Chuyển dữ liệu sang dạng quan hệ (long format)
df_longformat = pd.melt(df, id_vars=['Day'], var_name='Model', value_name='Score')
print("Dữ liệu dạng quan hệ:")
print(df_longformat)

Dữ liệu dạng quan hệ:
      Day Model  Score
0   Day 1     A    8.0
1   Day 2     A    7.5
2   Day 3     A    6.0
3   Day 4     A    7.0
4   Day 1     B    9.0
5   Day 2     B    8.5
6   Day 3     B    7.0
7   Day 4     B    6.0
8   Day 1     C    7.0
9   Day 2     C    7.0
10  Day 3     C    8.0
11  Day 4     C    5.0


In [60]:
# 2 Tạo bảng chéo để kiểm định chi-squared
# làm tròn điểm số để có dữ liệu rời rạc
df_longformat['Rounded_Score'] = df_longformat['Score'].round()

# Tạo bảng chéo giữa mẫu xe và điểm
contingency_table = pd.crosstab(df_longformat['Model'], df_longformat['Rounded_Score'])

# Thực hiện kiểm định chi-squared
chi2, p, dof, expected = chi2_contingency(contingency_table)

# In kết quả
print("\nBảng chéo (Contingency Table):")
print(contingency_table)
print("\nKết quả kiểm định chi-squared:")
print(f"Chi-squared statistic = {chi2:.4f}")
print(f"Degrees of freedom = {dof}")
print(f"P-value = {p:.4f}")
# Đánh giá
if p > 0.05:
    print("Bác bỏ ho => Không có sự khác biệt đáng kể giữa các mẫu xe")
else:
    print("Chấp nhận ho => Có sự khác biệt đáng kể giữa các mẫu xe dựa trên điểm số")


Bảng chéo (Contingency Table):
Rounded_Score  5.0  6.0  7.0  8.0  9.0
Model                                 
A                0    1    1    2    0
B                0    1    1    1    1
C                1    0    2    1    0

Kết quả kiểm định chi-squared:
Chi-squared statistic = 6.0000
Degrees of freedom = 8
P-value = 0.6472
Bác bỏ ho => Không có sự khác biệt đáng kể giữa các mẫu xe


In [61]:
# BÀI 3 Bảng flights(departure_time,…) chứa các giá trị thời gian dưới dạng số nguyên (ví dụ: 830 cho 8:30 AM, 1445 cho 2:45 PM).
#  Hãy chuyển đổi các giá trị này thành định dạng thời gian
import pandas as pd
from datetime import time

# Ví dụ bảng dữ liệu flights
data = {'departure_time': [830, 1445, 120,115,235]}
df = pd.DataFrame(data)

# Hàm chuyển đổi số nguyên thành dạng giờ:phút
def convert_to_time(value):
    if pd.isna(value):
        return None
    value = int(value)
    hour = value // 100
    minute = value % 100
    if 0 <= hour <= 23 and 0 <= minute <= 59:
        return time(hour, minute)
    else:
        return None  # loại trừ giá trị không hợp lệ

# Áp dụng chuyển đổi
df['departure_time_converted'] = df['departure_time'].apply(convert_to_time)
# Kết quả
print(df)

   departure_time departure_time_converted
0             830                 08:30:00
1            1445                 14:45:00
2             120                 01:20:00
3             115                 01:15:00
4             235                 02:35:00


In [62]:
# BÀI 4 Viết truy vấn SQL để tìm các ngoại lệ bằng cách sử dụng MAD. Một quy tắc chung là xem xét các giá trị ngoại lệ lớn hơn 1,5 lần so với giá trị MAD, trong đó x là số độ lệch chuẩn mà ta coi là có ý nghĩa
import pandas as pd
import numpy as np

# Ví dụ DataFrame
df = pd.DataFrame({
    'value': [4,11,5,6,8,7,9,80,64]  # có vài giá trị ngoại lệ
})

# Tính Median và MAD
median = df['value'].median()
mad = np.median(np.abs(df['value'] - median))

# Hệ số x = 1.5 
x = 1.5

# Đánh dấu các ngoại lệ
df['is_outlier'] = np.abs(df['value'] - median) > x * mad

# In kết quả
print("Giá trị trung vị (Median):", median)
print("MAD:", mad)
print("\nDữ liệu với đánh dấu ngoại lệ:")
print(df)


Giá trị trung vị (Median): 8.0
MAD: 3.0

Dữ liệu với đánh dấu ngoại lệ:
   value  is_outlier
0      4       False
1     11       False
2      5       False
3      6       False
4      8       False
5      7       False
6      9       False
7     80        True
8     64        True


In [63]:
# BÀI 5 Hãy xác định liệu hai người trong bảng Patient(last_name, weight, height) có phải là một người hay không bằng cách sử dụng khoảng cách kết hợp Boolean trên “last_name” và “weight”
import pandas as pd

# Ví dụ bảng Patient
data = {
    'last_name': ['Na', 'Nga', 'Tran', 'Na','Hoa','Mai'],
    'weight': [45, 50, 35, 45,45,65],
    'height': [170, 170, 170, 160,155,165]
}

df = pd.DataFrame(data)

# So sánh tất cả các cặp (tự ghép với chính nó)
from itertools import combinations

# Danh sách để lưu kết quả
matches = []

# Duyệt từng cặp
for i, j in combinations(df.index, 2):
    row_i = df.loc[i]
    row_j = df.loc[j]
    
    # So sánh Boolean
    same_last_name = row_i['last_name'] == row_j['last_name']
    same_weight = row_i['weight'] == row_j['weight']
    
    # Khoảng cách Boolean kết hợp
    score = int(same_last_name) + int(same_weight)
    
    if score == 2:  # đủ điều kiện là cùng một người
        matches.append((i, j))

# In kết quả
print("Các cặp được cho là cùng một người (dựa trên last_name và weight):")
for match in matches:
    print(f"Patient {match[0]} và Patient {match[1]}")

Các cặp được cho là cùng một người (dựa trên last_name và weight):
Patient 0 và Patient 3
